In [1]:
# Imports
import pandas as pd
import xarray as xr
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import plotly.io as pio
pio.renderers.default = "notebook_connected"

# Analyse de l'évolution des températures en France (1950-1974)

**Problématique :** Visualiser l'accélération du réchauffement climatique en France métropolitaine de 1950 à 1974 : des moyennes annuelles aux réalités locales.

Ce notebook utilise des données issues de la réanalyse ERA5 (Copernicus), masquées pour ne conserver que la France métropolitaine. 


# Chargement des données

In [ ]:
# Chargement du CSV (Historique Temporel)
df = pd.read_csv("donnees_meteo_france_25ans.csv")
df['time'] = pd.to_datetime(df['time'])
df['Year'] = df['time'].dt.year
df['Month'] = df['time'].dt.month

# Chargement du NetCDF (Carte)
try:
    ds_map = xr.open_dataset("carte_exemple_25ans.nc")
    # Conversion en DataFrame pour Plotly
    df_map = ds_map.to_dataframe().reset_index().dropna()
    # On s'assure d'avoir l'année pour l'animation
    if 'time' in df_map.columns:
        df_map['Year'] = df_map['time'].dt.year
    if df_map['t2m'].mean() > 200:
        df_map['Temperature_C'] = df_map['t2m'] - 273.15
    else:
        df_map['Temperature_C'] = df_map['t2m']
except Exception as e:
    print(f" Attention : Le fichier carte .nc n'a pas pu être chargé ({e})")
    df_map = None

# Courbe Anomalie Température France Métropolitaine (1950-1974)



In [9]:
# 1. Calcul de l'anomalie sur les 25 ans
ref_mean = df['Temperature_C'].mean()
df['Anomalie'] = df['Temperature_C'] - ref_mean

# 2. Agrégation par an
df_yearly = df.groupby('Year')[['Temperature_C', 'Anomalie']].mean().reset_index()

# 3. La Courbe (Line Chart)
fig1 = px.line(df_yearly, x='Year', y='Anomalie',
               markers=True, # Affiche les points
               title="Évolution de l'anomalie de température (1950-1974)",
               labels={'Anomalie': 'Écart à la moyenne (°C)'})


fig1.show()